In [78]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Input,Activation,Dropout
from tensorflow.keras.activations import sigmoid,relu
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
x = df.iloc[:,3:-1]
y =df.iloc[:,-1]

In [5]:
df['Geography'].unique()
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [6]:
### create dummy variables
geography = pd.get_dummies(x['Geography'],drop_first=True)
gender = pd.get_dummies(x['Gender'],drop_first=True)

In [7]:
### concatenating with the dataframe
x = pd.concat([x,geography,gender],axis=1)
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [8]:
### dropping the unnecessary columns
x = x.drop(['Geography','Gender'],axis=1)
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [9]:
### spliting the dataset into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [12]:
y_train.shape

(7000,)

In [15]:
### Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  =sc.fit_transform(X_test)

In [49]:
## perform hyperparameter optimization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential


In [91]:
## create a function for hyperparameter tunning
def create_model(layers,activation):
    model = Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.2))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.2))
            
    model.add(Dense(units=1,kernel_initializer = 'glorot_uniform',activation = 'sigmoid'))   
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model   

model = KerasClassifier(build_fn=create_model, verbose=0)

In [92]:
## given hyperparamter options to the model
layers=[[20],[40,20],[45,30,15]]
activations = ['sigmoid','relu']
## create a dictionary to save all the values
param_grids = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
param_grid

{'layers': [[20], [40, 20], [45, 30, 15]],
 'activation': ['sigmoid', 'relu'],
 'batch_size': [128, 256],
 'epochs': 30}

In [97]:
## to apply gridsearch cv for taking  differnt values for creating better model out of this options.that are gi
grid = GridSearchCV(estimator=model,param_grid = param_grids,cv=5)

In [98]:
grid_result = grid.fit(X_train,y_train)
grid_result

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000199A82319D0>,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [128, 256], 'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15]]})

In [100]:
print(grid_result.best_score_ , grid_result.best_params_)

0.8594285726547242 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [105]:
y_pred  =grid.predict(X_test)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [107]:
from sklearn.metrics import confusion_matrix,accuracy_score
cnn = confusion_matrix(y_pred,y_test)
cnn

array([[2282,  326],
       [  97,  295]], dtype=int64)

In [108]:
accuracy = accuracy_score(y_pred,y_test)
accuracy

0.859